In [ ]:
import tensorflow as tf 
import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input , MobileNetV2
import tensorflow_datasets as tfds 


## load the dataset 

In [ ]:
train_dataset  = tfds.load('cats_vs_dogs' , split = 'train[:75%]' , as_supervised = True)
val_dataset  =tfds.load('cats_vs_dogs' , split='train[90%:]' , as_supervised = True)

## load the model 

In [ ]:
pretrained_model = MobileNetV2(input_shape=(224,224,3) , weights='imagenet' , include_top=False) 


In [ ]:
for layer in pretrained_model.layers[:-5] : 
    layer.trainable=False


## process the dataset

In [ ]:
def preprocess_images(image  , label) : 
    image = image / 255
    image = tf.image.resize(image , (224,224)) 
    image = preprocess_input(image)
    return image  , label 

In [ ]:
train_dataset = train_dataset.map(preprocess_images).batch(64)

In [ ]:
val_dataset = val_dataset.map(preprocess_images).batch(64)

## build the model 

In [ ]:
inputs = tf.keras.layers.Input((224,224,3)) 
X = pretrained_model(inputs)
X = tf.keras.layers.Flatten()(X)
X = tf.keras.layers.Dense(265  ,activation= 'relu')(X)
X = tf.keras.layers.Dropout(.5)(X)
X= tf.keras.layers.Dense(1 , activation='sigmoid') (X)
model = tf.keras.Model(inputs  , X)


In [ ]:
model.compile(optimizer='adam' , loss = 'binary_crossentropy' , metrics= ['accuracy'])

In [ ]:
history = model.fit(train_dataset , epochs= 5 , validation_data=val_dataset ,batch_size=64)

## convert the model 


In [ ]:
model.save('cats_dogs_v1')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('cats_dogs_v1')
converter.optimizations = [tf.lite.Optimize.DEFAULT]



In [ ]:
tflite_model = converter.convert()
tflite_model_file = 'converted_model_v1.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

## test tflite model

In [ ]:
test_dataset = tfds.load('cats_vs_dogs' , split='train[80%:82%]' , as_supervised= True)

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
predictions = []

test_labels, test_imgs = [], []
for img, label in test_dataset.take(10):
    img = tf.image.resize(img , (224,224))
    img = img / 255
    img = tf.cast(img , tf.float32)
    img = tf.expand_dims(img ,0)
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    test_labels.append(label)
    test_imgs.append(img)